[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

Welcome to the Covid-19 Dashboard. This Dashboard has been built using the Public Health England API.

In [12]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [13]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [14]:
#Loads JSON files and store the raw data in some variable.
with open("vaccineseries.json", "r") as INFILE:
    jsondata=json.load(INFILE)

In [15]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d") #creates a function to convert date string to pandas datetime object

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe. """
    datalist=rawdata['data'] #retrieves list of dictionaries under the data key and assigns to datalist variable
    dates=[dictionary['date'] for dictionary in datalist ] #extracts all the dates from the dictionary and assigns to dates variable
    dates.sort() #sorts the date in 
    startdate=parse_date(dates[0]) #first item in the list
    enddate=parse_date(dates[-1]) #last item in the list
    index=pd.date_range(startdate, enddate, freq='D') #creates a date range, from the start to the end, with a frequency of each date being a day
    vaccinedf=pd.DataFrame(index=index, columns=['cases', 'reinfections', 'vaccinations']) #specifies its index and the title of the columns
    for entry in datalist: #each entry is a dictionary with date, cases, reinfections and vaccinations
        date=parse_date(entry['date']) #calls the parse_date function
        for column in ['cases', 'reinfections', 'vaccinations']: #list comprehension looping over each column
            #.isna checks the pd(parse_date) for missing values
            if pd.isna(vaccinedf.loc[date, column]): #.loc accesses a specific location in the dataframe (in this case, each loop)
                value= float(entry[column]) if entry[column]!=None else 0.0 #replaces None with 0
                vaccinedf.loc[date, column]=value #index,column in []
    vaccinedf.fillna(0.0, inplace=True) #if .isna returned True, fill in missing dates with 0.0
    return vaccinedf

vaccinedf=wrangle_data(jsondata) #df is the dataframe

## Download current data

Click the button below to refresh the data from Public Health England.

In [16]:
#here is a function which accesses the API for fresh data
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
    "areaType=nation", #filters areaType by nation
    "areaName=England" #filters areaName by England
    ]
    structure = {
    "date": "date", 
    "cases": "cumCasesByPublishDate", 
    "reinfections": "cumReinfectionsBySpecimenDate", 
    "vaccinations": "cumPeopleVaccinatedCompleteByVaccinationDate",
    }
    api = Cov19API(filters=filters, structure=structure) #calls the Cov19API with filters and structure and assigns it to api variable
    vaccineseries=api.get_json() #creates a json file from the api
    return vaccineseries #return data read from the API

In [17]:
#function for the api button to access the api
def api_button_callback(button):
    """ Button callback - Accesses API, wrangles data, updates global variable df used for plotting. """
    apidata=access_api() #calls the access_api function to provide current data
    global vaccinedf
    vaccinedf=wrangle_data(apidata) #calls wrangle_data function and applies to the apidata which was previously called, and assigns to dataframe
    refresh_graph()
    apibutton.icon="check"

#the refresh data widget
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='info',
    tooltip='Click to download current Public Health England data',
    icon='download',
)

apibutton.on_click(api_button_callback) #when the button is clicked, calls the api to refresh the data

display(apibutton)

Button(button_style='info', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click t…

## Graphs and Analysis

The graph below compares the number of cases, reinfections and vaccinations in England.

Please select a statistic. If you would like to select more than 1, please Ctrl-Click the statistics in the box.

If you would like to change the scale of the graph, please choose between the toggle buttons.

In [18]:
#SelectMultiple widget
series=wdg.SelectMultiple(
    options=['cases', 'reinfections', 'vaccinations'],
    value=['cases', 'reinfections', 'vaccinations'],
    rows=3,
    description='Stats:',
    disabled=False,
)    

#ToggleButtons widget
scale=wdg.ToggleButtons(
    options=['Linear', 'Log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-
    description='Scale:',
    disabled=False,
    button_style='info',
    tooltips=['Linear scale', 'Log scale'],
)

#function to change the scale of the graph from log to linear
def vaccineseries_graph(gcols, gscale):
    if gscale=='Linear':
        logscale=False #if gscale=linear, turn off log
    else:
        logscale=True #else keep it on
    ncols=len(gcols)
    if ncols>0:
        vaccinedf[list(gcols)].plot(logy=logscale, linewidth=2, linestyle='dashed') #plots the graph
        plt.show()
    else:
        print("Please choose a stat") 
        print("(CTRL-Click to select more than one category)") #print these lines if user does not choose a statistic

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   

#function to refresh the graph
def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. """
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other # forces the redraw
    scale.value=current # now we can change it back

ctrls=wdg.VBox([series, scale]) #puts controls into a box
display(ctrls)
graph=wdg.interactive_output(vaccineseries_graph, {'gcols': series, 'gscale': scale}) #draws the graph and adds the interactive widgets
form=wdg.HBox([graph, ctrls]) #puts graph and controls in a box
display(form) #displays the graph and controls

**Author and Copyright Notice** 2022 Created by Sarah Wakeley, with code from Fabrizio Smeraldi (f.smeraldi@qmul.ac.uk - web), all rights reserved. Data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*